## Notes From Transcript Example

Given a transcript (and an optional agenda), can we create a team that
will pull out a meeting summary, action items, attendees, and organized notes that are suitable
to be sent as meeting minutes to participants?

This code example will assume that the transcript and agenda are saved in local files and
will use the CrewAI [FileReadTool](https://docs.crewai.com/tools/FileReadTool/) so that agents can
access the information as needed.

**TODO** - Add the optional agenda to the notetaking task

### Lessons Learned
 * If you pass along a tool to an agent, the agent will use the tool at the slightest hint that it needs to be used! I had to explicitly add instructions in tasks when I passed information in a context array and to **not** try to read it from a file.
 * Tasks will only get information from the previous step. You can use the **context** array to pass the results from other previous steps.
 * The context is not labeled at all. So I had to ask agents to label their work so that it could be combined in the end.
 * The instructions [From the CrewAI Source Code](https://github.com/crewAIInc/crewAI/blob/main/src/crewai/translations/en.json) state "use the tools available and give your best Final Answer, your job depends on it!". This works *against* using chain of thought (asking agents to think step by step), and encourages the use of tools...

### Importing tools
First, let's import key libraries and tools:

In [1]:
import os
import textwrap
from crewai import Agent, Task, Crew, Process
from crewai_tools import FileReadTool

### Set the OpenAI connection and model

TODO: Create a .env file and use python-dotenv to load it

In [2]:
# NOTE set OPENAI_API_KEY in the environment to your OpenAI Key
# print(os.environ["OPENAI_API_KEY"])

# os.environ["OPENAI_MODEL_NAME"]="gpt-4o-mini"
os.environ["OPENAI_MODEL_NAME"]="gpt-4o"

### Initiate Tools and File Location

In [3]:
# Initiate the File Read Tool and set the location of the agenda
file_read_tool = FileReadTool()

transcript_location = "./transcript.txt"

### Defining the Individual Agents
Defining agents requires descriptive text for roles, general goals and backstory. We can also pass along Python functions (crewai tools) that
the agents can use to solve problems.

Here we'll define an expert (to come up with ideas), a writer (to write prose), and an editor (to make suggestions).

Notice the use of "{topic}" in the strings? These are not replaced now (these are not formatted string literals). Instead 
we will specify what goes in the "{topic}" sections when we kick off the crew down below.

In [4]:
# Creating the notetaker agent
notetaker = Agent(
    role='NoteTaker',
    goal='To convert a transcript into a usable set of notes (minutes) for meeting participants',
    backstory=textwrap.dedent("""
        You are a knowledgeable professional with a college education and extensive experience in the IT industry 
        as a project manager and writer. Your communication skills are excellent, and you understand the importance 
        of meeting best practices such as setting clear agendas, defining action items with specific responsibilities, 
        providing concise summaries, and maintaining documentation for future reference.
        
        Your role is to attentively listen to discussions and extract meaningful insights, ensuring that all 
        relevant points are captured for the benefit of meeting participants. You aim to create an inviting and 
        collaborative atmosphere, fostering an environment where everyone can contribute and thrive.
    """),
    tools=[file_read_tool]
)

### Now to define the tasks
Agents are assigned tasks, and tasks are organized into a workflow.

So to write our minutes, lets come up with a simple workflow:
* Read - Read the transcript from a file
* Identify Participants
* Identify Action Items
* Write Notes (fitting the agenda if there is one)
* Write meeting summary
* Combine and write final meeting minutes

In [5]:
# Identify meeting participants
participants_task = Task(
    description=textwrap.dedent("""
        Using the transcript from the file {transcript_location}, please identify and list all participants who attended the meeting.
        It's essential to include everyone who contributed to the discussion, as their input is valuable for documentation. 
        Please organize your response with the heading 'Attendees', followed by a bulleted list of names. 
        Format your response as follows:
        
        # Attendees
        - John Doe
        - Jane Smith
        - Alex Johnson
    """),
    expected_output='A formatted list of meeting participants',
    agent=notetaker,
)

# Identify action items
action_items_task = Task(
    description=textwrap.dedent("""
        Using the transcript from the file {transcript_location}, please provide a complete list of action items 
        identified in the meeting, including the name of the person assigned to each action item. 
        Capturing action items is crucial for ensuring accountability and follow-up.
        
        Please organize your response with the heading 'Action Items', followed by a numbered list. Each item should include 
        the name of the assignee and a brief summary of the action item. 
        If possible, include any deadlines or follow-up dates mentioned during the meeting.
        
        Format your response as follows:
        
        # Action Items
        1. Robert Buccigrossi - Prepare manager’s meeting notes for the next quarterly meeting (Due: [insert date here])
        2. Jane Smith - Review project deliverables and provide feedback by next week.
    """),
    expected_output='A formatted list of meeting action items',
    agent=notetaker,
)

# Identify notes
notes_task = Task(
    description=textwrap.dedent("""
        Using the transcript from the file {transcript_location}, please create a outline of the conversation organized
        by topic, with all the key subtopics, debates, observations, and decisions made during discussions under that
        topic. You will find it easier to focus if you break the transcript into smaller subsections, probably by
        major topics that were discussed.
        
        Please format your notes with the heading 'Meeting Notes', followed by clearly defined sections
        for different parts of the meeting. Each section should have a subheading, and the content should be organized
        in bullet points for clarity.
        
        Include the Meeting Title and Date at the top of the notes and use the following structure:
        
        # Meeting Notes
        
        **Meeting Title: [Insert Title Here]**
        **Date: [Insert Date Here]**
        
        ## Topic Name #1
        - Discution point, observation, decision, or subtopic
          - Sub discussion point, observation, or decision
          - ...
        - Discution point, observation, decision, or subtopic
          - Sub discussion point, observation, or decision
          - ...
        
        ## Topic Name #2
        - ...
        
        ## Topics Name #3
        - ...
        
    """),
    expected_output='A detailed, structured outline of the meeting notes in bullet points',
    agent=notetaker,
)

# Write meeting summary
summary_task = Task(
    description=textwrap.dedent("""
        Using the notes provided, summarize the overall purpose and key points of the meeting. Your summary should be in 
        prose format (one or two paragraphs) and should cover the main objectives, significant discussions, key decisions made, 
        and any notable outcomes. 
        
        The notes should be sufficient to create this summary, so you WILL NOT NEED to read the transcript directly from a file.
        
        Please begin the summary with the heading 'Meeting Summary'.
        
        Here is an example structure:
        
        # Meeting Summary
        
        The meeting focused on [main objective], where participants discussed [key topics discussed].
        Significant decisions included [key decisions], and it was agreed that [summary of outcomes].
        Notable contributions were made by [names], who [specific contributions].
    """),
    expected_output='A summary (one or two paragraphs) of the meeting in prose format',
    agent=notetaker,
)

# Combine into a meeting minutes
combine_task = Task(
    description=textwrap.dedent("""
        Please combine the elements from the previous workflow steps into a single set of meeting minutes text to be shared with the team: 
        Attendees, Meeting Summary, Action Items, and Meeting Notes. All necessary information has been provided in the context, so 
        you WILL NOT NEED to read the transcript directly from a file.
        
        Please format the meeting minutes in markdown using the following structure, making sure to place the Meeting Title and Date at the top:
        
        # Minutes [Insert Meeting Title Here] - [Insert Date Here]
        
        ## Meeting Summary
        [Summary of the meeting]
        
        ## Action Items
        [List of action items]
        
        ## Attendees
        [List of attendees]
        
        ## Agenda
        - List of key topics
        
        ## Topic Name #1
        - ...
        
        ## Topic Name #2
        - ...
        
        ## Topics Name #3
        - ...
        
        ## Decisions Made
        - Specific decisions or outcomes reached during the meeting.
        
        ## Follow-up
        - Any follow-up actions that arose, without listing the attendees or action items directly.
        
        Ensure that each section is clearly labeled and formatted consistently. The goal is to create a comprehensive 
        and professional meeting minutes document that is easy to read and follow.

        Since the results will be directly displayed to the user, please present the Markdown 
        directly (not in a Markdown block).
    """),
    expected_output='Meeting minutes formatted in markdown',
    context=[participants_task, action_items_task, notes_task, summary_task],
    agent=notetaker,
)


### Form and kick off the crew

We kick off the crew, assigning the transcript_location into the variable "transcript_location".

In [6]:
# Forming the crew and defining the process
crew = Crew(
    agents=[notetaker],
    tasks=[participants_task, action_items_task, notes_task, summary_task, combine_task],
    process=Process.sequential,
    verbose=True,
)


# Kicking off the crew
result = crew.kickoff(inputs={'transcript_location': transcript_location})


 [2024-10-27 15:05:41][DEBUG]: == Working Agent: NoteTaker
 [2024-10-27 15:05:41][INFO]: == Starting Task: 
Using the transcript from the file ./transcript.txt, please identify and list all participants who attended the meeting.
It's essential to include everyone who contributed to the discussion, as their input is valuable for documentation. 
Please organize your response with the heading 'Attendees', followed by a bulleted list of names. 
Format your response as follows:

# Attendees
- John Doe
- Jane Smith
- Alex Johnson

 


May 30, 2012 Meeting Transcript

June 20, 2012

UNITED STATES OF AMERICA\
DEPARTMENT OF COMMERCE

COMMERCE SPECTRUM MANAGEMENT\
ADVISORY COMMITTEE (CSMAC)

MEETING

WEDNESDAY\
MAY 30, 2012

The Advisory Committee met in Room 4830, Herbert C. Hoover Building, 1401 Constitution Avenue, N.W., Washington, D.C., at 10:00 a.m., Brian Fontes, Chair, presiding.

Members Present:

Dr. Brian Fontes, Chief Executive Officer,\
Â Â Â Â Â Â Â Â  National Emergency Number Ass

In [7]:
# Print the final result
print(result)

# Minutes CSMAC Meeting - May 30, 2012

## Meeting Summary
The CSMAC meeting held on May 30, 2012, was centered around the strategic objective of enhancing spectrum sharing between commercial entities and government agencies. Assistant Secretary Strickling and Tom Power from the White House underscored the necessity of spectrum sharing due to increasing demand and limited resources. Karl Nebbia presented a report on the 1755-1850 MHz band, highlighting the identification of 115 MHz for sharing, the challenges of diverse operations, and the substantial relocation costs estimated at $18 billion. The discussion led to a proposal for forming five working groups aimed at facilitating the transition to commercial wireless broadband. These groups will focus on areas such as weather satellite receivers, law enforcement surveillance systems, regulatory constructs, tactical radio systems, and airborne operations, with co-chairs from both government and industry.

The meeting also established the